<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
# Imports from today's lecture
import numpy as np
import gensim
import os
import re
import spacy

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

C:\Users\ethan\Anaconda3\envs\U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Amazon Data

In [2]:
path = '../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'

In [3]:
df = pd.read_csv(path)
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


### Text Preprocessing

In [4]:
nlp = spacy.load("en_core_web_lg")

In [5]:
def tokenize(text):
    "Complete this function"
    
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [6]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [7]:
df['tokens'][0][0:10]

['order',
 'item',
 'bad',
 'quality',
 'missing',
 'backup',
 'spring',
 'pcs',
 'aluminum',
 'battery']

### Gensim LDA Topic Modeling

In [8]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(df['tokens'])

In [9]:
# Testing for id from word
id2word.token2id['battery']

3

In [10]:
# See length of tokens
len(df['tokens'])

28332

In [11]:
# Remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [12]:
# Create a corpus or bag of words
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [13]:
# Checking to make sure I did it right
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [14]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 5,
                   passes=10,
                   workers=8
                  )

In [15]:
# Viewing my topics
lda.print_topics()

[(0,
  '0.079*"great" + 0.031*"tablet" + 0.024*"price" + 0.022*"product" + 0.017*"works" + 0.016*"good" + 0.012*"recommend" + 0.012*"use" + 0.012*"screen" + 0.011*"amazon"'),
 (1,
  '0.051*"tablet" + 0.024*"use" + 0.021*"loves" + 0.021*"kids" + 0.020*"easy" + 0.019*"great" + 0.018*"old" + 0.016*"bought" + 0.016*"games" + 0.016*"year"'),
 (2,
  '0.048*"kindle" + 0.023*"love" + 0.013*"great" + 0.012*"best" + 0.012*"read" + 0.011*"use" + 0.011*"screen" + 0.011*"bought" + 0.010*"new" + 0.010*"buy"'),
 (3,
  '0.085*"batteries" + 0.044*"good" + 0.033*"price" + 0.032*"great" + 0.022*"long" + 0.021*"work" + 0.015*"amazon" + 0.014*"battery" + 0.014*"brand" + 0.013*"buy"'),
 (4,
  '0.018*"bought" + 0.016*"amazon" + 0.016*"tablet" + 0.014*"price" + 0.014*"better" + 0.012*"books" + 0.009*"great" + 0.008*"good" + 0.008*"black" + 0.008*"buy"')]

### Selecting an appropriate number of topics

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

#### The Coherence values cell took way too long to run and I had to drop it

In [ ]:
# Making topic coherence
topic_coherence = pd.DataFrame.from_records(coherence_values)
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

### Visualization

In [16]:
# Import pyLDAvis and enable notebook
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [17]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\ethan\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.144404  0.053745       1        1  28.281399
3     -0.206904  0.011846       2        1  28.132196
2      0.038206 -0.079337       3        1  17.412941
0      0.024377 -0.093127       4        1  16.250437
4     -0.000082  0.106873       5        1   9.923024, topic_info=     Category         Freq       Term        Total  loglift  logprob
18    Default  7569.000000  batteries  7569.000000  30.0000  30.0000
2076  Default  3012.000000     kindle  3012.000000  29.0000  29.0000
21    Default  9312.000000      great  9312.000000  28.0000  28.0000
2433  Default  6830.000000     tablet  6830.000000  27.0000  27.0000
1857  Default  2147.000000      loves  2147.000000  26.0000  26.0000
17    Default  5208.000000      price  5208.000000  25.0000  25.0000
165   Default  2133.000000    product  2133.000000  24.0000  24.0000
20    Default  1272.000000      brand  1272.000000  23.0000  23.0000
652   Default  1694.000000      games  1694.000000  22.0000  22.0000
23    Default  2424.000000       long  2424.000000  21.0000  21.0000
34    Default  5968.000000       good  5968.000000  20.0000  20.0000
9     Default  2280.000000       work  2280.000000  19.0000  19.0000
1548  Default  1790.000000       apps  1790.000000  18.0000  18.0000
2087  Default  1506.000000      books  1506.000000  17.0000  17.0000
409   Default  1121.000000       play  1121.000000  16.0000  16.0000
192   Default  2450.000000       kids  2450.000000  15.0000  15.0000
28    Default  3202.000000     bought  3202.000000  14.0000  14.0000
96    Default  1523.000000      works  1523.000000  13.0000  13.0000
105   Default  1752.000000       year  1752.000000  12.0000  12.0000
90    Default  3485.000000       love  3485.000000  11.0000  11.0000
2     Default  2118.000000    battery  2118.000000  10.0000  10.0000
1679  Default  1621.000000     screen  1621.000000   9.0000   9.0000
521   Default  2101.000000        old  2101.000000   8.0000   8.0000
19    Default  1455.000000     better  1455.000000   7.0000   7.0000
74    Default   837.000000     brands   837.000000   6.0000   6.0000
755   Default  2912.000000       easy  2912.000000   5.0000   5.0000
786   Default  1186.000000       read  1186.000000   4.0000   4.0000
7     Default  1374.000000    quality  1374.000000   3.0000   3.0000
166   Default  1125.000000  recommend  1125.000000   2.0000   2.0000
387   Default  1067.000000      value  1067.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
19     Topic5   417.551605     better  1455.691528   1.0615  -4.2916
115    Topic5   159.712006       know   434.037292   1.3106  -5.2527
67     Topic5   142.176346       ones   383.445343   1.3182  -5.3690
2087   Topic5   357.219666      books  1506.062500   0.8714  -4.4477
28     Topic5   552.176025     bought  3202.217529   0.5526  -4.0122
31     Topic5   219.893631  christmas   895.213745   0.9064  -4.9329
11     Topic5   144.161575  expensive   476.932861   1.1139  -5.3551
291    Topic5    77.701126     couldn   188.927628   1.4218  -5.9732
1823   Topic5   144.043304   grandson   506.055450   1.0538  -5.3559
466    Topic5   221.402267     little  1003.421143   0.7991  -4.9261
24     Topic5   490.403412     amazon  3811.629150   0.2597  -4.1308
217    Topic5   146.589081     charge   559.375732   0.9711  -5.3384
235    Topic5   204.860504     device  1041.421875   0.6843  -5.0037
946    Topic5   206.721588    reading  1223.958618   0.5318  -4.9947
254    Topic5   158.821411      money   751.593079   0.7559  -5.2583
101    Topic5   207.713150        got  1252.557129   0.5135  -4.9899
17     Topic5   433.952423      price  5208.720215  -0.1748  -4.2531
2433   Topic5   486.852966     tablet  6830.467773  -0.3309  -4.1381
106    Topic5   170.771118       need   943.231323   0.6013  -5.1857
29     Topic5   231.602905        buy  2267.812012   0.0288 

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling